Le système de recommandation est un outil de filtrage de l'information qui cherche à prédire quel produit un utilisateur va aimer, et sur cette base, recommande quelques produits aux utilisateurs. Par exemple, Amazon peut recommander de nouveaux articles à acheter, Netflix peut recommander de nouveaux films à regarder et Google peut recommander des actualités susceptibles d'intéresser un utilisateur. Les deux approches les plus utilisées pour construire un système de recommandation sont le filtrage basé sur le contenu (CBF) et le filtrage collaboratif (CF).

L'objectif du système de recommandation est de prédire les évaluations de ces éléments. Ensuite, les éléments les mieux notés peuvent être recommandés aux utilisateurs respectifs. Dans les problèmes du monde réel, on s'attend à ce que la matrice d'utilité soit très éparse, car chaque utilisateur ne rencontre qu'une petite fraction d'éléments parmi le vaste ensemble d'options disponibles

In [1]:
! pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 46.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=3e918736b075d1be3f56a1077915e5f2373cd026e6da36fd40fd7876f6abb9e0
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

In [3]:
from pyspark import SparkContext
sc=SparkContext()

Pour charger les données sous forme de dataframe spark, importez pyspark et instanciez une session spark.

In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('abc').getOrCreate()

Il y a 2 tuples, movies et ratings qui contiennent des variables telles que MovieID::Genre::Title et UserID::MovieID::Rating::Timestamp respectivement.


1.   Charger les données dans un RDD chaque ligne est sous forme d'une chaine de caractère apres on va spliter chaque ligne par une tab et on cast chaque case selon le casting ci dessous apres on tronsforme la RDD en dataframe car ALS comprend just le DATAFRAME




In [5]:
lines = sc.textFile("/content/sample_data/u.data")
parts = lines.map(lambda row: row.split("\t"))#séparer les colonnes par␣ tabulation
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),rating=float(p[2]), timestamp=int(p[3])))
#transformer␣,→chaque ligne du RDD "parts" à un objet Row qui contient des colonnes avec␣,
#→leurs noms et leurs types correspondants

print(ratingsRDD.first())
ratings = spark.createDataFrame(ratingsRDD)#créer le dataframe

['196', '242', '3', '881250949']
Row(userId=196, movieId=242, rating=3.0, timestamp=881250949)


2.  On va appliquer le coldstrat=drop pour sumprimer tout les users qui n'ont pas donner un rating,il va les identifier et les supprimer de meme pour movies
3. Apres nous allons appliquer le param Grid Builder() pour qu'il boucle dans rank et iter et choisi le meilleiur choix

In [7]:
(training, test) = ratings.randomSplit([0.8, 0.2])
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", nonnegative= True)
#séparer aleatoirement l'ensemble d'entrainement et l'ensemble de test respectivement selon les pourcentages 80% et 20%
param_grid=ParamGridBuilder().addGrid(als.rank,[12,13,14]).addGrid(als.maxIter,[5,6,7]).addGrid(als.regParam,[.1,.2,.3]).build()
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
tvs= TrainValidationSplit(estimator=als, estimatorParamMaps=param_grid,evaluator=evaluator)
print ("start training")
model = tvs.fit(training)


start training


4. pour tester la performance du modele nous allons appliquer le rmse qui permet de calculer l'écart entre les valeurs réels et prédite

In [8]:
best_model=model.bestModel
print ("start testing")
predictions = best_model.transform(test)
print ("start evaluation")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))
print("**bestModel**")
print(" rank:"), best_model.rank
print(" maxIter:"), best_model._java_obj.parent().getMaxIter()
print(" regParam:"), best_model._java_obj.parent().getRegParam()
predictions.sort("userId", "rating").show()

start testing
start evaluation
Root-mean-square error = 0.9258704267262458
**bestModel**
 rank:
 maxIter:
 regParam:
+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|    219|   1.0|878542327|  3.157984|
|     1|    104|   1.0|875241619| 1.7559676|
|     1|    103|   1.0|878542845|   1.55215|
|     1|    247|   1.0|875241619|0.91633904|
|     1|    233|   2.0|878542552|  3.096055|
|     1|     36|   2.0|875073180| 1.7918693|
|     1|    225|   2.0|878542738| 2.1766071|
|     1|     27|   2.0|876892946| 3.0114648|
|     1|    105|   2.0|875240739| 2.2090845|
|     1|    102|   2.0|889751736|  3.024571|
|     1|    205|   3.0|878542909| 4.2347946|
|     1|    117|   3.0|874965739|   3.38307|
|     1|      4|   3.0|876893119| 3.7962031|
|     1|    217|   3.0|876892676| 3.0883493|
|     1|     24|   3.0|875071713|  3.544331|
|     1|    232|   3.0|878543196| 3.2465258|
|     1|     49|   3.0|87854

RMSE=0.92 ce qui vaut dire que sur une base  5 ratings le modèle commis un écart presque 1

# diminuer rmse en changeant les hyperparametrs

In [17]:
(training, test) = ratings.randomSplit([0.85, 0.15])
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", nonnegative= True)
param_grid=ParamGridBuilder().addGrid(als.rank,[8,9,14]).addGrid(als.maxIter,[5,6,10]).addGrid(als.regParam,[.1,.2,.3]).build()
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
tvs= TrainValidationSplit(estimator=als, estimatorParamMaps=param_grid,evaluator=evaluator)
print ("start training")
model = tvs.fit(training)

start training


In [18]:
best_model=model.bestModel
print ("start testing")
predictions = best_model.transform(test)
print ("start evaluation")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))
print("**bestModel**")
print(" rank:"), best_model.rank
print(" maxIter:"), best_model._java_obj.parent().getMaxIter()
print(" regParam:"), best_model._java_obj.parent().getRegParam()
predictions.sort("userId", "rating").show()

start testing
start evaluation
Root-mean-square error = 0.9189294795969207
**bestModel**
 rank:
 maxIter:
 regParam:
+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|    260|   1.0|875071713| 2.5553405|
|     1|    120|   1.0|875241637|  1.797494|
|     1|    131|   1.0|878542552| 3.9151368|
|     1|    263|   1.0|875693007| 2.1135724|
|     1|     74|   1.0|889751736| 2.9592407|
|     1|    149|   2.0|878542791| 3.4974918|
|     1|    237|   2.0|875071749| 3.6188843|
|     1|    118|   3.0|875071927|  2.621757|
|     1|    164|   3.0|876893171| 3.4595132|
|     1|     70|   3.0|875072895| 3.7134166|
|     1|    217|   3.0|876892676|  3.216447|
|     1|     69|   3.0|875072262| 3.6321673|
|     1|     65|   4.0|875072125|  3.777763|
|     1|     28|   4.0|875072173| 3.6751468|
|     1|    133|   4.0|876892818| 3.4920053|
|     1|    222|   4.0|878873388| 3.3912282|
|     1|     80|   4.0|87689